In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_common_lib
importlib.reload(p3_common_lib)
from p3_common_lib import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
                                    get_varying_column_names, filter_dataframe, take_varying_columns,
                                    load_json_records_as_dataframe)

In [ ]:
# Load result files from P3 Test Driver
src_files = []
src_files += ['../data/p3_test_driver/results/*.json']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
def parse_images_per_sec(output):
    """Search for last occurance of: total images/sec: 8169.88"""
    try:
        for m in re.finditer('^total images/sec: ([.0-9]+)$', output, flags=re.MULTILINE):
            pass
        return float(m.groups()[0])
    except:
        return np.nan

In [ ]:
# Clean raw results
def clean_result(result):
    try:
        r = result.copy()
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        r['images_per_sec'] = parse_images_per_sec(r['output'])
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['np']
        r['storage_type'] = 'isilon'
        return pd.Series(r)
    except Exception as e:
        print('ERROR: %s: %s' % (r['test_uuid'], e))
        # raise e

In [ ]:
#r = clean_result(raw_df.iloc[-1])
#pd.DataFrame(r)

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.set_index('test_uuid', drop=False)
clean_df = clean_df[clean_df.error==False]
clean_df = clean_df.sort_values(['utc_begin'])

In [ ]:
# Show list of columns
list(clean_df.columns.values)

In [ ]:
raw_other_results_df = pd.read_csv('other_benchmark_results.csv')
raw_other_results_df.info()

In [ ]:
# Clean raw results
def clean_other_result(result):
    try:
        r = result.copy()        
        r['images_per_sec_per_gpu'] = r['images_per_sec'] / r['np']
        r['NVIDIA_TENSORFLOW_VERSION'] = str(r['NVIDIA_TENSORFLOW_VERSION'])
        return pd.Series(r)
    except Exception as e:
        #print('ERROR: %s: %s' % (r['test_uuid'], e))
        raise e

In [ ]:
clean_other_results_df = raw_other_results_df.apply(clean_other_result, axis=1)
clean_other_results_df.T

In [ ]:
clean_other_results_df.NVIDIA_TENSORFLOW_VERSION.drop_duplicates().values

In [ ]:
clean_df = pd.concat([clean_df, clean_other_results_df], sort=False)

In [ ]:
clean_df.NVIDIA_TENSORFLOW_VERSION.drop_duplicates()

In [ ]:
# Define columns that identify test parameters
param_cols = [
 'NVIDIA_TENSORFLOW_VERSION',
 'batch_group_size',
 'batch_size',
 'cached',
 'datasets_num_private_threads',
 'datasets_prefetch_buffer_size',
 'fp16',
 'model',
 'np',
 'num_batches',
 'num_inter_threads',
 'num_intra_threads',
 'tensorflow_benchmark_git_hash',
 'test']

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'utc_begin',    
    'images_per_sec',
    'images_per_sec_per_gpu',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
# Export to CSV
clean_df[cols].to_csv('results.csv')

In [ ]:
clean_df.groupby(['model']).size()

In [ ]:
# First level of filtering
filt_df = filter_dataframe(
    clean_df,
#     cached=False,
)
len(filt_df)

In [ ]:
filt1_df = filter_dataframe(
    filt_df,
    storage_type='isilon',
)
len(filt1_df)

In [ ]:
df = filt_df.groupby([
    'NVIDIA_TENSORFLOW_VERSION',
    'fp16',
    'model',
    'np',
    'storage_type',
    'cached',
]).mean()[['images_per_sec_per_gpu']]
df

In [ ]:
df = df.unstack(['storage_type','cached'])
df

In [ ]:
#baseline = df[('images_per_sec_per_gpu','isilon',True)]
baseline = df[('images_per_sec_per_gpu','na',False)]

In [ ]:
(df.div(baseline, axis=0) - 1.0) * 100.0